In [1]:
from manim import *
from numpy import linalg as npl
import matplotlib.pyplot as plt
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline

video_scene = " -v WARNING --progress_bar None --disable_caching moi_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s moi_Scene"

Manim Community v0.16.0.post0

Jupyter Capture Output v0.0.6


In [8]:
%%capture_video --path "animations/moment_of_inertia/moment_of_inertia.mp4"
%%manim -ql --fps 20 $video_scene

class moi_Scene(ThreeDScene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)


        # Equations
        moi_eq1 = MathTex(r'\Theta=\int r_{\perp}^2 dm', font_size = 36).align_on_border(UP + LEFT, buff = 0.5)
        moi_eq2 = MathTex(r'\Theta=\varrho\int_V r_{\perp}^2 dV', font_size = 36).align_on_border(UP + LEFT, buff = 0.5)
        moi_eq3 = MathTex(r'\mathrm{mit}~\varrho=const.=\frac{m}{V}', font_size = 36, color = BLUE).next_to(moi_eq2, 2*RIGHT)     

        cone_eq1 = MathTex(r'\Xi:=\{(x, y, z)\in\mathbb{R}:x^2+y^2\leq R^2,\,0\leq z\leq H-2R\}', font_size = 36).align_on_border(LEFT + DOWN).shift(0.5*UP)
        cone_eq2 = Tex(r'$\mathrm{Kegel}$ $\Xi$: Höhe $H$, Radius $R(z)=(H-z)/2$', font_size = 24, color = GREY).align_on_border(LEFT + DOWN)


        # 3D-Coordinate System
        CO3D = [-6, 0, -6]
        ax = ThreeDAxes(
            x_range = (-3, 3, 1), y_range = (-3, 3, 1), z_range = (-2, 5, 1),
            x_length = 6, y_length = 6, z_length = 6, axis_config = {'tip_length': 0.05, 'tip_width': 0.3}, 
            z_axis_config = {'color': WHITE},
            ).move_to(CO3D).rotate(
                axis = [1, 0, 0], angle = 6*PI/4
                ).rotate(
                    axis = [0, 1, 0], angle = 5*PI/4
                    ).rotate(axis = [1, 0, 0], angle = PI/12).rotate(
                        axis = [0, 1, 0], angle = PI/16
                        )#.add_coordinates()


        # Cone
        def cone_circ(z):
            r = (4 - z) / 2
            return ax.plot_parametric_curve(lambda phi: np.array([r*np.cos(phi), r*np.sin(phi), z]), t_range = [0, 2*PI], stroke_opacity = 0.125, color = GREY)

        cone_circ_list = [cone_circ(z) for z in np.arange(0, 4.01, 0.1)]


        # Mass-Elements
        def cone_cube(x, y, z):
            dm_cube = Cube(side_length = 0.125, fill_opacity = 0, color = RED, stroke_width = 1).rotate(
                    axis = [1, 0, 0], angle = 6*PI/4
                    ).rotate(
                        axis = [0, 1, 0], angle = 5*PI/4
                        ).rotate(axis = [1, 0, 0], angle = PI/12).rotate(
                            axis = [0, 1, 0], angle = PI/16
                            ).move_to(ax.coords_to_point(x, y, z))
            dm_radius = Line(ax.coords_to_point(x, y, z), ax.coords_to_point(0, 0, z), color = RED, stroke_width = 1)
            dm = VGroup(dm_cube, dm_radius)
            dV_cube = Cube(side_length = 0.125, fill_opacity = 0, color = BLUE, stroke_width = 1).rotate(
                    axis = [1, 0, 0], angle = 6*PI/4
                    ).rotate(
                        axis = [0, 1, 0], angle = 5*PI/4
                        ).rotate(axis = [1, 0, 0], angle = PI/12).rotate(
                            axis = [0, 1, 0], angle = PI/16
                            ).move_to(ax.coords_to_point(x, y, z))
            dV_radius = Line(ax.coords_to_point(x, y, z), ax.coords_to_point(0, 0, z), color = BLUE, stroke_width = 1)
            dV = VGroup(dV_cube, dV_radius)
            return dm, dV


        cone_cube1 = cone_cube(3/2, -1/2, 0.5)
        cone_cube2 = cone_cube(0, 1.5, 1.5)
        #cone_cube3 = cone_cube(1/2, -1/2, 2)



        self.add(moi_eq1)
        self.add(cone_eq1, cone_eq2)
        self.add(ax)
                
        for cone_circ_i in cone_circ_list:
            self.wait(0.05)
            self.add(cone_circ_i)

        self.wait(0.5)
        self.play(Create(cone_cube1[0]), run_time = 1.5)
        self.play(Create(cone_cube2[0]), run_time = 1.5)
        self.wait(0.5)
        self.play(Write(moi_eq3))
        self.wait(0.5)
        self.play(Transform(moi_eq1, moi_eq2), Transform(cone_cube1[0], cone_cube1[1]), Transform(cone_cube2[0], cone_cube2[1]), run_time = 1.5)
        self.wait(0.5)
        self.play(Uncreate(moi_eq3))
        #self.play(Create(cone_cube3[0]))
        #self.wait(1.5)
        #self.play(FadeTransform(cone_cube1[0], cone_cube1[1]), FadeTransform(cone_cube2[0], cone_cube2[1]), FadeTransform(cone_cube3[0], cone_cube3[1]))
        self.wait(3)
            